<a href="https://colab.research.google.com/github/HedersonSantos/Noticias/blob/main/preProcessamento_noticias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
from google.colab import files
import io
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('max_colwidth', None)

### Upload da base de noticias

In [4]:
#csvNoticias = files.upload()  
!wget https://raw.githubusercontent.com/HedersonSantos/Noticias/main/noticia_tratada.zip
!unzip noticia_tratada.zip
%ls



--2021-05-29 19:59:21--  https://raw.githubusercontent.com/HedersonSantos/Noticias/main/noticia_tratada.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4676937 (4.5M) [application/zip]
Saving to: ‘noticia_tratada.zip’

noticia_tratada.zip 100%[===================>]   4.46M  26.1MB/s    in 0.2s    

2021-05-29 19:59:21 (26.1 MB/s) - ‘noticia_tratada.zip’ saved [4676937/4676937]

Archive:  noticia_tratada.zip
replace noticia_tratada.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: noticia_tratada.csv     
noticia_tratada.csv  noticia_tratada.zip  sample_data/


In [107]:
#dfNews_all = pd.read_csv(io.BytesIO(csvNoticias['noticia_tratada.csv']), header=0, delimiter=';')
dfNews_all = pd.read_csv('noticia_tratada.csv', header=0, delimiter=';')
print(dfNews_all.columns)
dfNews_all.shape

Index(['FONTE', 'CATEGORIA', 'TEXTO', 'URL', 'PUBLISHEDAT', 'TITLE', 'AUTHOR',
       'DT_REGISTRO', 'TEXT_TRATADO'],
      dtype='object')


(3001, 9)

### Tratamento da CATEGORIA

In [108]:
print('Dimensao Inicial:',dfNews_all.shape)
#Elimina noticias duplicadas
dfNews = dfNews_all.drop_duplicates(subset=['URL'],keep='first').copy()
#Elimina noticias sem texto
dfNews = dfNews[(~dfNews['TEXT_TRATADO'].isnull())]
print('Dimensão após duplicadas e textos em branco:',dfNews.shape)
#Tira as marcações do campo Text[] do Postgres
dfNews['CATEGORIA'] = dfNews.loc[:,['CATEGORIA']].apply(lambda x: x['CATEGORIA'].replace('[','').replace(']','').replace('"','').replace(", 'redacao'",'').replace("'",''),axis=1)
#Agrupa as notícias locais como categoria LOCAL
siglaEstados = ['ac','al','ap','am','ba','ce','df','es','go','ma','mt','ms','mg','pa','pb','pr','pe','pi','rj','rn','rs','ro','rr','sc','sp','se','to']
nomeEstados = ['acre','alagoas','amapa','amazonas','bahia','ceara','distrito-federal','espirito-santo','goias','maranhao','mato-grosso','mato-grosso-sul','minas-gerais','minas',
               'para','paraiba','parana','pernambuco','piaui','rio-janeiro','rio-grande-norte','rio-grande-sul','rondonia','roraima','santa-catarina','sao-paulo','sergipe','tocantins']
nomeCidades =['rio','bh',]
local = siglaEstados + nomeEstados + nomeCidades
dfNews['CATEGORIA'] = dfNews.loc[:,['CATEGORIA']].apply(lambda x: 'local' if x['CATEGORIA'] in local else x['CATEGORIA'],axis=1)
#efe, afp, rfi são notícias internacionais. 
mundo = ['efe','afp','rfi']
dfNews['CATEGORIA'] = dfNews.loc[:,['CATEGORIA']].apply(lambda x: 'mundo' if x['CATEGORIA'] in mundo else x['CATEGORIA'],axis=1)
#Na lista miscelania há as categorias que possuem miscelânias de noticias de outras categorias.
miscelania = ['deutschewelle', '','bbc','ansa','universa']
dfNews['CATEGORIA'] = dfNews.loc[:,['CATEGORIA']].apply(lambda x: 'miscelania' if x['CATEGORIA'] in miscelania else x['CATEGORIA'],axis=1)
#'reuters' é uma categoria especifica do UOL. Possui textos de economia e miscelânia de noticias.
dfreuters = dfNews[dfNews['CATEGORIA']=='reuters'].copy()
dfreuters['CATEGORIA'] = dfreuters.loc[:,['CATEGORIA','URL']].apply(lambda x: 'economia' if 'economia' in x['URL']  else 'miscelania',axis=1)
dfNews=dfNews[dfNews['CATEGORIA']!='reuters']
dfNews = pd.concat([dfNews,dfreuters],sort=False)
#noticias é uma categoria da globo. Traz noticias do Jornal Nacional. São miscelânias.
dfnoticia = dfNews[dfNews['CATEGORIA']=='noticia'].copy()
dfnoticia['CATEGORIA'] = dfnoticia.loc[:,['CATEGORIA','URL']].apply(lambda x: 'saude' if 'ciencia-e-saude' in x['URL']  else 'miscelania',axis=1)
dfNews=dfNews[dfNews['CATEGORIA']!='noticia']
dfNews = pd.concat([dfNews,dfnoticia],sort=False)
#redacao é uma categoria da uol. Traz noticias do meio-ambiente. VERIFICAR QUANDO ATUALIZAR O DATASET SE PERMANECE ASSIM.
dfNews['CATEGORIA'] = dfNews.loc[:,['CATEGORIA','URL']].apply(lambda x: 'meio-ambiente' if ('meio-ambiente' in x['URL']) and
                                                              (x['CATEGORIA']=='redacao') else x['CATEGORIA'],axis=1)
dfNews['CATEGORIA'] = dfNews.loc[:,['CATEGORIA','URL']].apply(lambda x: 'meio-ambiente' if ('meio-ambiente' in x['URL']) and
                                                              (x['CATEGORIA']=='sociedade') else 
                                                              'miscelania' if x['CATEGORIA']=='sociedade' else x['CATEGORIA'] ,axis=1)
dfNews['CATEGORIA'] = dfNews.loc[:,['CATEGORIA']].apply(lambda x: 'colunas' if x['CATEGORIA']=='vivabem, colunas' else x['CATEGORIA'] ,axis=1)
dfNews['CATEGORIA'] = dfNews.loc[:,['CATEGORIA']].apply(lambda x: 'esporte' if x['CATEGORIA']=='futebol' else x['CATEGORIA'],axis=1)
dfNews['CATEGORIA'] = dfNews.loc[:,['CATEGORIA']].apply(lambda x: 'midia' if x['CATEGORIA']=='Famosos' else x['CATEGORIA'],axis=1)
dfNews.shape

Dimensao Inicial: (3001, 9)
Dimensão após duplicadas e textos em branco: (2937, 9)


(2937, 9)

In [109]:
#Filtra categorias com poucas noticias. 
dfNewsGrupo = dfNews.groupby(['CATEGORIA'])['URL'].count().reset_index().sort_values('URL',ascending=False)
dfNewsGrupo = dfNewsGrupo[dfNewsGrupo['URL']<=10]
dfNews = dfNews[~dfNews['CATEGORIA'].isin(dfNewsGrupo['CATEGORIA'].tolist())]
dfNews.shape

(2884, 9)